In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score,recall_score
from sklearn.metrics import f1_score,roc_auc_score,roc_curve
from sklearn.model_selection import train_test_split


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cascade-cup-22/sample_submission.csv
/kaggle/input/cascade-cup-22/train.csv
/kaggle/input/cascade-cup-22/test.csv


In [2]:
def generate_model_report(y_actual,y_predcited):
    print("Accuracy = ", accuracy_score(y_actual,y_predcited))
    print("Precision = ", precision_score(y_actual,y_predcited,zero_division=1))
    print("Recall = ", recall_score(y_actual,y_predcited))
    print("F1 score = ", f1_score(y_actual,y_predcited))
    pass

In [3]:
data = pd.read_csv("/kaggle/input/cascade-cup-22/train.csv")
test_data = pd.read_csv("/kaggle/input/cascade-cup-22/test.csv")

In [4]:
data.head()

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
0,2021-01-26 02:21:35,556753,2021-01-26 00:00:00,2021-01-26 02:21:59,2021-01-26 02:22:08,2021-01-26 02:32:51,2021-01-26 02:49:47,11696,1.5666,2.65,46.0,46.0,0,0.0,621.0,NaN,NaN,NaN,NaN,NaN
1,2021-01-26 02:33:16,556754,2021-01-26 00:00:00,2021-01-26 02:33:57,2021-01-26 02:34:45,2021-01-26 02:50:25,2021-01-26 03:11:15,18117,2.5207,2.76,8.0,8.0,0,0.0,105.0,NaN,NaN,NaN,3.266667,NaN
2,2021-01-26 02:39:49,556755,2021-01-26 00:00:00,2021-01-26 02:39:57,2021-01-26 02:40:13,2021-01-26 02:56:00,2021-01-26 03:12:46,18623,2.2074,4.80,1.0,1.0,0,0.0,66.0,NaN,NaN,NaN,9.816667,NaN
3,2021-01-26 02:47:53,556756,2021-01-26 00:00:00,2021-01-26 02:48:25,2021-01-26 02:49:06,2021-01-26 03:21:51,2021-01-26 03:41:05,15945,2.1894,6.38,1.0,1.0,0,0.0,127.0,NaN,NaN,NaN,17.533333,NaN
4,2021-01-26 03:06:30,556757,2021-01-26 00:00:00,2021-01-26 03:07:21,2021-01-26 03:07:57,2021-01-26 03:31:38,2021-01-26 04:00:15,17589,2.7870,4.01,34.0,34.0,0,0.0,84.0,NaN,NaN,NaN,1.350000,NaN


In [ ]:
data['order_time'] = pd.to_datetime(data['order_time'])

data['order_time_year'] = data['order_time'].dt.year
data['order_time_month'] = data['order_time'].dt.month
data['order_time_week'] = data['order_time'].dt.week
data['order_time_day'] = data['order_time'].dt.day
data['order_time_hour'] = data['order_time'].dt.hour
data['order_time_minute'] = data['order_time'].dt.minute
data['order_time_dayofweek'] = data['order_time'].dt.dayofweek

In [ ]:
data['allot_time'] = pd.to_datetime(data['allot_time'])

#data['order_time_year'] = data['order_time'].dt.year
data['allot_time_month'] = data['allot_time'].dt.month
data['allot_time_week'] = data['allot_time'].dt.week
data['allot_time_day'] = data['allot_time'].dt.day
data['allot_time_hour'] = data['allot_time'].dt.hour
data['allot_time_minute'] = data['allot_time'].dt.minute
data['allot_time_dayofweek'] = data['allot_time'].dt.dayofweek

In [ ]:
data.drop(['reassignment_method','reassignment_reason','reassigned_order','cancelled_time','order_time','order_date','allot_time'], axis=1, inplace=True)

In [ ]:
data.isna().sum()

In [ ]:
data.drop(['session_time','delivered_time','pickup_time','accept_time'], axis=1, inplace=True)

In [ ]:
data['lifetime_order_count'].mean()

In [ ]:
data['undelivered_orders'] = data['undelivered_orders'].fillna(data['undelivered_orders'].mean())
data['delivered_orders'] = data['delivered_orders'].fillna(data['delivered_orders'].mean())
data['alloted_orders'] = data['alloted_orders'].fillna(data['alloted_orders'].mean())
data['lifetime_order_count'] = data['lifetime_order_count'].fillna(data['lifetime_order_count'].mean())

In [ ]:
data.isna().sum()

In [ ]:
data.nunique()

In [ ]:
X = data.drop("cancelled",axis=1)
y = data["cancelled"]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [ ]:
ax = sns.countplot(x =data['cancelled'] )
print(data['cancelled'].value_counts())

In [ ]:
print(100* (5218/float(data.shape[0])))
print(100* (444782/float(data.shape[0])))

In [ ]:
from sklearn.linear_model import LogisticRegression 
clf =LogisticRegression().fit(X_train,y_train)

In [ ]:
y_test_pred = clf.predict(X_test)

In [ ]:
generate_model_report(y_test,y_test_pred)

UNDERSAMPLING

In [ ]:
minority_class_len = len(data[data['cancelled']==1])
print(minority_class_len)

In [ ]:
majority_class_indices = data[data['cancelled']==0].index
print(majority_class_indices)

In [ ]:
random_majority_indices = np.random.choice(majority_class_indices,minority_class_len,replace=False)
print(len(random_majority_indices))

In [ ]:
minority_class_indices = data[data['cancelled']==1].index
print(minority_class_indices)

In [ ]:
under_sample_indices = np.concatenate([minority_class_indices,random_majority_indices])

In [ ]:
under_sample = data.loc[under_sample_indices]

In [ ]:
sns.countplot(x='cancelled',data = under_sample)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
X = under_sample.drop("cancelled",axis=1)
y = under_sample["cancelled"]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33,random_state=42)
model = RandomForestClassifier()
n_estim=range(100,1000,100)

## Search grid for optimal parameters
param_grid = {"n_estimators" :n_estim}


model_rf = GridSearchCV(model,param_grid = param_grid, cv=5, scoring="accuracy", n_jobs= 4, verbose = 1)

model_rf.fit(X_train,y_train)
print(model_rf.best_score_)

#best estimator
model_rf.best_estimator_

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
X = under_sample.drop("cancelled",axis=1)
y = under_sample["cancelled"]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33,random_state=42)
model =LinearDiscriminantAnalysis()
param_grid = {'tol':[0.001,0.0011,0.01,0.02,.1,.2]}

modell = GridSearchCV(model,param_grid = param_grid, cv=5, scoring="precision", n_jobs= 4, verbose = 1)

modell.fit(X_train,y_train)

# Best score
print(modell.best_score_)

# Best Estimator
modell.best_estimator_

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
model = LogisticRegression()
n_estim=range(10 ,200,10)
param_grid = {'C':n_estim}
modell = GridSearchCV(model,param_grid = param_grid, cv=5, scoring="accuracy", n_jobs= 4, verbose = 1)
modell.fit(X_train,y_train)
print(modell.best_estimator_)

# Best score
print(modell.best_score_)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
X = under_sample.drop("cancelled",axis=1)
y = under_sample["cancelled"]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33,random_state=42)
model = Ridge()
ridge_params = {'alpha':[200, 230, 250,265, 270, 275, 290, 300, 500]}
modelr = GridSearchCV(model,param_grid = ridge_params, cv=5, scoring="accuracy", n_jobs= 4, verbose = 1)
modelr.fit(X_train,y_train)
print(modelr.best_estimator_)

# Best score
print(modelr.best_score_)



In [ ]:
#from sklearn.model_selection import GridSearchCV
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.svm import SVC
#from sklearn.naive_bayes import GaussianNB
#from sklearn.ensemble import AdaBoostClassifier
X = under_sample.drop("cancelled",axis=1)
y = under_sample["cancelled"]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=0)
#from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge
clf =Ridge(alpha =220,random_state=10).fit(X_train,y_train)
y_test_pred = clf.predict(X_test)

In [ ]:
#from sklearn.neighbors import KNeighborsClassifier 
#clf =KNeighborsClassifier(n_neighbors=25).fit(X_train,y_train)
#y_test_pred = clf.predict(X_test)
generate_model_report(y_test,y_test_pred)

In [ ]:
test_data = pd.read_csv("/kaggle/input/cascade-cup-22/test.csv")

In [ ]:
test_data.isna().sum()

In [ ]:
test_data['order_time'] = pd.to_datetime(test_data['order_time'])

test_data['order_time_year'] =test_data['order_time'].dt.year
test_data['order_time_month'] = test_data['order_time'].dt.month
test_data['order_time_week'] =test_data['order_time'].dt.week
test_data['order_time_day'] = test_data['order_time'].dt.day
test_data['order_time_hour'] =test_data['order_time'].dt.hour
test_data['order_time_minute'] = test_data['order_time'].dt.minute
test_data['order_time_dayofweek'] = test_data['order_time'].dt.dayofweek

In [ ]:
test_data['allot_time'] = pd.to_datetime(test_data['allot_time'])

test_data['allot_time_month'] = test_data['allot_time'].dt.month
test_data['allot_time_week'] =test_data['allot_time'].dt.week
test_data['allot_time_day'] = test_data['allot_time'].dt.day
test_data['allot_time_hour'] =test_data['allot_time'].dt.hour
test_data['allot_time_minute'] = test_data['allot_time'].dt.minute
test_data['allot_time_dayofweek'] = test_data['allot_time'].dt.dayofweek

In [ ]:
test_data.drop(['reassignment_method','reassignment_reason','reassigned_order','order_time','order_date','allot_time'], axis=1, inplace=True)

In [ ]:
test_data.isna().sum()

In [ ]:
test_data.drop(['session_time','accept_time'], axis=1, inplace=True)

In [ ]:
test_data['undelivered_orders'] = test_data['undelivered_orders'].fillna(test_data['undelivered_orders'].mean())
test_data['delivered_orders'] = test_data['delivered_orders'].fillna(test_data['delivered_orders'].mean())
test_data['alloted_orders'] = test_data['alloted_orders'].fillna(test_data['alloted_orders'].mean())
test_data['lifetime_order_count'] = test_data['lifetime_order_count'].fillna(test_data['lifetime_order_count'].mean())

In [ ]:
test_data.isna().sum()

In [ ]:
print(data.shape)
print(test_data.shape)

In [ ]:
test_preds =clf.predict(test_data)
output = pd.DataFrame({'order_id': test_data['order_id'],
                       'cancelled': test_preds})
output.to_csv('submission_last.csv', index=False)